### Download Dataset

In [3]:
import os
# put your kaggle ac username and kaggle key which you will get after generating api token
os.environ['KAGGLE_USERNAME'] = "**********"
os.environ["KAGGLE_KEY"] = "**********"

!kaggle datasets download paultimothymooney/chest-xray-pneumonia


 99% 2.27G/2.29G [00:24<00:00, 101MB/s]
100% 2.29G/2.29G [00:24<00:00, 101MB/s]


In [ ]:
!unzip /content/chest-xray-pneumonia.zip

### Import Libraries

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,  Flatten, Input, Lambda
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob


In [10]:
IMAGE_SIZE  = [224,224]

train_path = r"/content/chest_xray/train"
test_path = r"/content/chest_xray/test"


### download model

In [11]:
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top = False)

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
for layer in vgg.layers:
    layer.trainable = False

In [14]:
folders = glob(r"/content/chest_xray/train/*")
print(folders)


['/content/chest_xray/train/NORMAL', '/content/chest_xray/train/PNEUMONIA']

In [15]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs = vgg.input, outputs = prediction)

model.summary()

In [17]:
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']

)

### image augmentation to increase train and test data

In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
train_set = train_datagen.flow_from_directory(train_path,
                                              target_size = (224,224),
                                              batch_size = 64,
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224,224),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Fitting the model

In [23]:
r = model.fit_generator(
    train_set,
    validation_data = test_set,
    epochs = 1,
    steps_per_epoch = len(train_set),
    validation_steps = len(test_set)

)

<ipython-input-23-b0d78edad6f5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


82/82 [==============================] - 148s 2s/step - loss: 0.2742 - accuracy: 0.8903 - val_loss: 0.3360 - val_accuracy: 0.8830


### Saving the model

In [24]:
from keras.models import load_model

model.save('chest_xray.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### loading the saved model and testing it on new xray image

In [25]:
nModel = load_model('chest_xray.h5')

In [33]:
img = image.load_img(r"/content/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg",target_size = (224,224))


x = image.img_to_array(img)
print(x.shape)
x = np.expand_dims(x,axis = 0)
print(x.shape)
img_data = preprocess_input(x)

classes = nModel.predict(img_data)
print(classes)
result = classes[0]

if np.argmax(result) == 0:
    print("result is normal")
if  np.argmax(result) == 1:
    print("person is affected with penumonia")



(224, 224, 3)
(1, 224, 224, 3)
1/1 [==============================] - 0s 19ms/step
[[3.4546984e-14 1.0000000e+00]]
person is affected with penumonia
